In [84]:
import ultralytics
from ultralytics import YOLO
import argparse
import pytesseract
import cv2

In [ ]:
# This is the argument parser to help me detect what the users what the system to perform


def arg_parser():
    parse_input = argparse.ArgumentParser(
        description="This project is for detecting cars number plate"
    )
    # parse_input.add_argument(
    #     "--mode",
    #     type=str,
    #     # required=True,
    #     help="Please select test or train",
    #     choices=["train", "test"],
    # )
    # parse_input.add_argument("--image", type=str, help="Image")
    # parse_input.add_argument("--video", help="Image file only", type=str, )
    # parse_input.add_argument("epoch", type=int,)

    return parse_input.parse_args([])

In [ ]:
# models and training


def test_trainer(args):
    model = YOLO("yolov8n.pt")
    model.train(data=args.data, epochs=args.epochs, imgsz=args.images)

In [ ]:
if __name__ == "__main__":
    args = arg_parser()
    args.mode = 'train'
    args.epochs = 10
    args.data = 'datasets/data.yaml'
    args.images = 640
    if args.mode == 'train':
        test_trainer(args)
        

In [153]:
# saved_model = YOLO("runs/detect/train15/weights/best.pt")  # or last.pt
# saved_model.predict(source="screenshot2.png", save=True)

result = cv2.imread('runs/detect/predict6/screenshot2.jpg')

for box in r.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cropped = result[y1:y2, x1:x2]

    # Resize for better OCR
    # cropped = cv2.resize(cropped, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    # cv2.imwrite("debug_plate.jpg", cropped)


    # Grayscale and preprocess
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    denoised = cv2.bilateralFilter(gray,  d=11, sigmaColor=17, sigmaSpace=17)
    thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    cv2.imwrite("1_gray.jpg", gray)
    cv2.imwrite("2_denoised.jpg", denoised)
    cv2.imwrite("3_thresh.jpg", thresh)
    cv2.imwrite("4_morphed.jpg", morphed)
    # OCR
    text = pytesseract.image_to_string(morphed, config='--psm 7')
    print("🔠 Detected Plate Text:", text)

🔠 Detected Plate Text: 
